In [ ]:
print('ritu')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df_train = pd.read_csv('Smartphone/train.csv')
df_test = pd.read_csv('Smartphone/test.csv')
print(df_train.shape,df_test.shape)

In [ ]:
X_train = df_train.drop(['Activity','subject'],axis = 1)
y_train = df_train['Activity']
X_test = df_test.drop(['Activity','subject'],axis = 1)
y_test = df_test['Activity']


In [ ]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
# display(y_train_le,y_test_le)

In [ ]:
print(y_train_le.shape)
print(y_test_le.shape)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train,y_train_le)
predict = log_reg.predict(X_test)
accuracy = accuracy_score(y_test_le,predict)

In [ ]:
print('Test accuracy:',accuracy)

### 1. Removing Duplicate Columns

In [ ]:
def get_duplicate_columns(df):
    duplicate_columns = {}
    seen_columns = {}
    
    for column in df.columns:
        current_column = df[column]

        try:
            current_column_hash = current_column.values.tobytes()
        except Exception as e:
            current_column_hash = current_column.to_string().encode()

        if current_column_hash in seen_columns:
            if seen_columns[current_column_hash] in duplicate_columns:
                duplicate_columns[seen_columns[current_column_hash]].append(column)
            else:
                duplicate_columns[seen_columns[current_column_hash]] = [column]
        else:
            seen_columns[current_column_hash]  = column
    return duplicate_columns

In [ ]:
duplicates = get_duplicate_columns(X_train)
duplicates

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
for one_list in duplicates.values():
     X_train.drop(columns = one_list,inplace = True)
     X_test.drop(columns = one_list,inplace = True)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Variance Threshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vh = VarianceThreshold(threshold = 0.05)

In [ ]:
vh.fit(X_train)

In [ ]:
sum(vh.get_support())

In [ ]:
columns = X_train.columns[vh.get_support()]
columns

In [ ]:
X_train = vh.transform(X_train)
X_test = vh.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train,columns = columns)
X_test = pd.DataFrame(X_test,columns = columns)


In [ ]:
X_train.head()

In [ ]:
print(X_train.shape)
print(X_test.shape)



# Correlation

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(X_train.corr())

In [ ]:
corr_matrix = X_train.corr()
corr_matrix

In [ ]:
columns = corr_matrix.columns

columns_to_drop = []
for i in range(len(columns)):
    for j in range(i+1,len(columns)):
        if corr_matrix.loc[columns[i],columns[j]]>0.95:
            columns_to_drop.append(columns[j])



In [ ]:
print(len(columns_to_drop))

In [ ]:
columns_to_drop = set(columns_to_drop)

In [ ]:
len(columns_to_drop)

In [ ]:
X_train.drop(columns = columns_to_drop,axis = 1,inplace = True)
X_test.drop(columns = columns_to_drop,axis = 1,inplace = True)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# ANOVA

In [ ]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

skb = SelectKBest(f_classif,k=100)
skb.fit(X_train,y_train)

In [ ]:
columns = X_train.columns[skb.get_support()]
len(columns)

In [ ]:
X_train = skb.transform(X_train)
X_test = skb.transform(X_test)

X_train = pd.DataFrame(X_train,columns = columns)
X_test = pd.DataFrame(X_test,columns = columns)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Moment of Truth

In [ ]:
log_new = LogisticRegression(max_iter=1000)  # Increase max_iter if it doesn't converge
log_new.fit(X_train, y_train_le)

In [ ]:
predict_new = log_new.predict(X_test)
accuracy_new = accuracy_score(y_test_le,predict_new)
print('Old accuracy:',accuracy)
print('New accuracy:',accuracy_new)

# Chi-square Test

In [ ]:
titanic = pd.read_csv('titanic/train.csv',usecols = ['Survived','Pclass','Sex','SibSp','Parch','Embarked'])
titanic.head()

In [ ]:
ct = pd.crosstab(titanic['Survived'],titanic['Sex'],margins = True)
ct

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(ct)

In [ ]:
score = []

for column in titanic.columns[1:]:
    ct = pd.crosstab(titanic['Survived'],titanic[column])
    score.append(chi2_contingency(ct)[1])

In [ ]:
pd.Series(score,index = titanic.columns[1:]).sort_values().plot(kind = 'bar')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

le = LabelEncoder()
titanic_encoded = titanic.apply(le.fit_transform)

X = titanic_encoded.drop('Survived',axis = 1)
y = titanic_encoded['Survived']

chi2_scores = chi2(X,y)

In [ ]:
chi2_scores

In [ ]:
p_values = pd.Series(chi2_scores[1],index = X.columns).sort_values()
p_values

In [ ]:
p_values.plot.bar()
plt.title('Chi-square test - P-values')
plt.xlabel('Feature')
plt.ylabel('P-value')

plt.show()
